In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Store CSV into DataFrame

In [2]:
file1 = "Resources/PPPLoans.csv"
loans_df = pd.read_csv(file1)
loans_df.head(1)

,LoanRange,BusinessName,Address,City,State,Zip,NAICSCode,BusinessType,RaceEthnicity,Gender,Veteran,NonProfit,JobsRetained,DateApproved,Lender,CD
0,a $5-10 million,"ARCTIC SLOPE NATIVE ASSOCIATION, LTD.",7000 Uula St,BARROW,AK,99723.0,813920.0,Non-Profit Organization,Unanswered,Unanswered,Unanswered,Y,295.0,4/14/2020,"National Cooperative Bank, National Association",AK - 00


In [3]:
### Create new data with select columns

In [4]:
loans_new=loans_df[["State","JobsRetained"]]
loans_new.head(1)

,State,JobsRetained
0,AK,295.0


In [5]:
loans_state=loans_new.groupby("State").sum()
loans_state.head(1)

,JobsRetained
State,
AK,67587.0


In [6]:
loans_state.reset_index(inplace=True)
loans_state.head(1)

,State,JobsRetained
0,AK,67587.0


In [12]:
file2 = "Resources/June Unemployment Rate.csv"
unemp_rate_df = pd.read_csv(file2)
unemp_rate_df.head(1)

,State,June 2020 unemployment rate,June 2019 unemployment rate,12-month percentage point change
0,Massachusetts,17.4,2.9,14.5


In [13]:
june_unemp_rate_df = unemp_rate_df[["State","June 2020 unemployment rate"]]
june_unemp_rate_df.rename(columns={"June 2020 unemployment rate":"unemp_rate"},inplace = True)

In [30]:
june_rate_df=june_unemp_rate_df
june_rate_df.head(1)

,State,unemp_rate
0,Massachusetts,17.4


In [32]:
file3="Resources/state_name.csv"
state_name=pd.read_csv(file3)
state_name.rename(columns={"Abbreviation":"State","State":"State_Name"},inplace=True)
state_name=state_name[["State","State_Name"]]
state_name.head(1)

,State,State_Name
0,AL,ALABAMA


### Connect to local database

In [24]:
rds_connection_string = "postgres:6325460@localhost:5432/ppp_loans"
engine = create_engine(f'postgresql://{rds_connection_string}')

### Check for tables

In [25]:
engine.table_names()

['state_name', 'ppp_loan', 'unemployment_rate']

### Use pandas to load csv converted DataFrame into database

In [26]:
loans_state.to_sql(name='ppp_loan', con=engine, if_exists='replace', index=False)
june_rate_df.to_sql(name='unemployment_rate',con=engine, if_exists='replace', index=False)
state_name.to_sql(name='state_name', con=engine, if_exists='replace', index=False)

### Confirm data has been added by querying the customer_name table

In [27]:
pd.read_sql_query('select * from ppp_loan', con=engine).head()

,State,JobsRetained
0,AK,67587.0
1,AL,408803.0
2,AR,209212.0
3,AS,1565.0
4,AZ,616020.0


In [28]:
pd.read_sql_query('select * from state_name', con=engine).head()

,State,State_Name
0,AL,ALABAMA
1,AK,ALASKA
2,AZ,ARIZONA
3,AR,ARKANSAS
4,CA,CALIFORNIA


In [29]:
pd.read_sql_query('select * from unemployment_rate', con=engine).head()

,State,unemp_rate
0,Massachusetts,17.4
1,New Jersey,16.6
2,New York,15.7
3,Nevada,15.0
4,California,14.9
